# Recommender Systems - CatBoost Ranker

Learning to Rank с CatBoost:
- YetiRank / YetiRankPairwise
- PairLogitPairwise
- Ranking метрики (NDCG, MAP)
- Feature engineering для ранкинга
- Group-wise обучение

In [ ]:
!pip install catboost pandas numpy scikit-learn -q

In [ ]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import ndcg_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

print("✓ Библиотеки загружены!")

## 1. Загрузка данных

In [ ]:
# === ВАШИ ДАННЫЕ ===
# Формат для ранкинга: query_id (или user_id), item_id, relevance (или rating), features
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Названия колонок
QUERY_COL = 'user_id'  # или 'query_id'
ITEM_COL = 'item_id'
RELEVANCE_COL = 'rating'  # или 'relevance', 'clicks', etc.

print(f"Train samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"\nУникальных queries (users): {train_df[QUERY_COL].nunique()}")
print(f"Уникальных items: {train_df[ITEM_COL].nunique()}")
print(f"\nПример данных:")
print(train_df.head())

## 2. Feature Engineering для ранкинга

In [ ]:
def create_ranking_features(df, query_col, item_col, relevance_col):
    """
    Создание признаков для ранкинга
    """
    df = df.copy()
    
    # 1. Query-level признаки
    # Количество айтемов у query
    query_item_count = df.groupby(query_col).size().to_dict()
    df['query_item_count'] = df[query_col].map(query_item_count)
    
    # Средний relevance для query
    query_avg_relevance = df.groupby(query_col)[relevance_col].mean().to_dict()
    df['query_avg_relevance'] = df[query_col].map(query_avg_relevance)
    
    # Максимальный relevance для query
    query_max_relevance = df.groupby(query_col)[relevance_col].max().to_dict()
    df['query_max_relevance'] = df[query_col].map(query_max_relevance)
    
    # 2. Item-level признаки
    # Популярность айтема (сколько раз встречается)
    item_popularity = df.groupby(item_col).size().to_dict()
    df['item_popularity'] = df[item_col].map(item_popularity)
    
    # Средний relevance для айтема
    item_avg_relevance = df.groupby(item_col)[relevance_col].mean().to_dict()
    df['item_avg_relevance'] = df[item_col].map(item_avg_relevance)
    
    # Количество уникальных queries для айтема
    item_query_count = df.groupby(item_col)[query_col].nunique().to_dict()
    df['item_query_count'] = df[item_col].map(item_query_count)
    
    # 3. Query-Item interaction признаки
    # Относительный relevance (relevance / max_relevance в группе)
    df['relative_relevance'] = df[relevance_col] / (df['query_max_relevance'] + 1e-5)
    
    # Z-score relevance внутри query
    query_std_relevance = df.groupby(query_col)[relevance_col].std().to_dict()
    df['query_std_relevance'] = df[query_col].map(query_std_relevance).fillna(0)
    df['relevance_zscore'] = (
        (df[relevance_col] - df['query_avg_relevance']) / (df['query_std_relevance'] + 1e-5)
    )
    
    # 4. Ranking-специфичные признаки
    # Позиция айтема в отсортированном списке query (по relevance)
    df['rank_position'] = df.groupby(query_col)[relevance_col].rank(ascending=False, method='first')
    
    # Нормализованная позиция (0-1)
    df['rank_position_norm'] = df['rank_position'] / df['query_item_count']
    
    return df

# Применение
print("Генерация ranking признаков...")
train_df = create_ranking_features(train_df, QUERY_COL, ITEM_COL, RELEVANCE_COL)

print(f"✓ Признаки созданы!")
print(f"Количество признаков: {len(train_df.columns)}")
print(f"\nНовые признаки:")
new_features = [col for col in train_df.columns if col not in [QUERY_COL, ITEM_COL, RELEVANCE_COL]]
print(new_features)

## 3. Encoding categorical признаков

In [ ]:
# Label encoding для query и item IDs
query_encoder = LabelEncoder()
item_encoder = LabelEncoder()

# Fit на всех данных
all_queries = pd.concat([train_df[QUERY_COL], test_df[QUERY_COL]]).unique()
all_items = pd.concat([train_df[ITEM_COL], test_df[ITEM_COL]]).unique()

query_encoder.fit(all_queries)
item_encoder.fit(all_items)

train_df['query_encoded'] = query_encoder.transform(train_df[QUERY_COL])
train_df['item_encoded'] = item_encoder.transform(train_df[ITEM_COL])

print("✓ Encoding завершен")

## 4. Подготовка данных для CatBoost Ranker

In [ ]:
# Выбор признаков
feature_cols = [
    'query_encoded', 'item_encoded',
    'query_item_count', 'query_avg_relevance', 'query_max_relevance',
    'item_popularity', 'item_avg_relevance', 'item_query_count',
    'relative_relevance', 'relevance_zscore',
    'rank_position', 'rank_position_norm'
]

# Добавляем все остальные числовые признаки если есть
other_numeric = train_df.select_dtypes(include=[np.number]).columns.tolist()
for col in other_numeric:
    if col not in feature_cols and col not in [QUERY_COL, ITEM_COL, RELEVANCE_COL]:
        feature_cols.append(col)

X = train_df[feature_cols]
y = train_df[RELEVANCE_COL]
groups = train_df['query_encoded']  # Group ID для ранкинга

print(f"Признаков для обучения: {len(feature_cols)}")
print(f"Групп (queries): {groups.nunique()}")
print(f"Samples: {len(X)}")

## 5. Train/Val split с сохранением групп

In [ ]:
# Разбиваем по query groups, а не по отдельным samples
unique_queries = train_df['query_encoded'].unique()
train_queries, val_queries = train_test_split(
    unique_queries, test_size=0.2, random_state=42
)

train_mask = train_df['query_encoded'].isin(train_queries)
val_mask = train_df['query_encoded'].isin(val_queries)

X_train = X[train_mask]
y_train = y[train_mask]
groups_train = groups[train_mask]

X_val = X[val_mask]
y_val = y[val_mask]
groups_val = groups[val_mask]

print(f"Train: {len(X_train)} samples, {groups_train.nunique()} queries")
print(f"Val: {len(X_val)} samples, {groups_val.nunique()} queries")

## 6. Создание CatBoost Pool с группами

In [ ]:
# CatBoost Pool для ranking
train_pool = cb.Pool(
    data=X_train,
    label=y_train,
    group_id=groups_train  # Важно для ranking!
)

val_pool = cb.Pool(
    data=X_val,
    label=y_val,
    group_id=groups_val
)

print("✓ CatBoost Pools созданы!")

## 7. Обучение CatBoost Ranker

In [ ]:
# Параметры для ranking
ranker_params = {
    'iterations': 2000,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'YetiRank',  # Можно также: 'YetiRankPairwise', 'PairLogitPairwise'
    'eval_metric': 'NDCG',  # Можно также: 'PFound', 'MAP'
    'random_seed': 42,
    'verbose': 200,
    'early_stopping_rounds': 100,
    'task_type': 'GPU' if cb.cuda.is_cuda_available() else 'CPU'
}

print(f"Обучение CatBoost Ranker с {ranker_params['loss_function']}...\n")

ranker = cb.CatBoost(ranker_params)
ranker.fit(
    train_pool,
    eval_set=val_pool,
    use_best_model=True,
    verbose=200
)

print("\n✓ CatBoost Ranker обучен!")

## 8. Оценка модели

In [ ]:
# Предсказания на валидации
val_predictions = ranker.predict(val_pool)

# Вычисление NDCG для каждой query
def compute_ndcg_per_query(y_true, y_pred, groups, k=10):
    """
    Вычисление NDCG@k для каждой группы
    """
    ndcg_scores = []
    
    for group_id in np.unique(groups):
        mask = groups == group_id
        group_true = y_true[mask]
        group_pred = y_pred[mask]
        
        if len(group_true) > 1:
            # NDCG требует 2D массивы
            ndcg = ndcg_score([group_true], [group_pred], k=min(k, len(group_true)))
            ndcg_scores.append(ndcg)
    
    return np.mean(ndcg_scores)

# NDCG@10
ndcg_10 = compute_ndcg_per_query(
    y_val.values, 
    val_predictions, 
    groups_val.values, 
    k=10
)

print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ВАЛИДАЦИИ")
print("="*60)
print(f"NDCG@10: {ndcg_10:.4f}")
print(f"Best iteration: {ranker.get_best_iteration()}")
print("="*60)

## 9. Feature Importance

In [ ]:
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': ranker.get_feature_importance()
}).sort_values('importance', ascending=False)

print("\nТоп-15 важных признаков:")
print(feature_importance.head(15))

# Визуализация
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.barh(feature_importance.head(15)['feature'], feature_importance.head(15)['importance'])
plt.xlabel('Importance')
plt.title('Топ-15 важных признаков для ранкинга')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 10. Анализ топ предсказаний

In [ ]:
# Создаем DataFrame с предсказаниями
val_results = train_df[val_mask].copy()
val_results['predicted_score'] = val_predictions

# Для одного query смотрим топ-10 предсказанных айтемов
sample_query = val_results['query_encoded'].iloc[0]
query_results = val_results[val_results['query_encoded'] == sample_query].sort_values(
    'predicted_score', ascending=False
).head(10)

print(f"\nТоп-10 предсказаний для query {sample_query}:")
print(query_results[[ITEM_COL, RELEVANCE_COL, 'predicted_score', 'rank_position']].to_string(index=False))

## 11. Подготовка test данных

In [ ]:
# Применяем ту же генерацию признаков
# Но используем статистику из train!

def create_test_features(test_df, train_df, query_col, item_col):
    """
    Создание признаков для test используя статистику из train
    """
    test = test_df.copy()
    
    # Строим маппинги из train
    item_popularity = train_df.groupby(item_col).size().to_dict()
    item_avg_relevance = train_df.groupby(item_col)[RELEVANCE_COL].mean().to_dict()
    item_query_count = train_df.groupby(item_col)[query_col].nunique().to_dict()
    
    # Применяем к test
    test['item_popularity'] = test[item_col].map(item_popularity).fillna(0)
    test['item_avg_relevance'] = test[item_col].map(item_avg_relevance).fillna(
        train_df[RELEVANCE_COL].mean()
    )
    test['item_query_count'] = test[item_col].map(item_query_count).fillna(0)
    
    # Query-level признаки (считаем по test, т.к. это айтемы для одного запроса)
    test['query_item_count'] = test.groupby(query_col)[item_col].transform('count')
    
    # Encoding
    test['query_encoded'] = query_encoder.transform(test[query_col])
    test['item_encoded'] = item_encoder.transform(test[item_col])
    
    # Заполняем остальные признаки нулями или средними
    for col in feature_cols:
        if col not in test.columns:
            test[col] = 0
    
    return test

print("Подготовка test данных...")
test_df = create_test_features(test_df, train_df, QUERY_COL, ITEM_COL)

X_test = test_df[feature_cols]
groups_test = test_df['query_encoded']

test_pool = cb.Pool(
    data=X_test,
    group_id=groups_test
)

print(f"✓ Test данные готовы: {len(X_test)} samples, {groups_test.nunique()} queries")

## 12. Предсказания на test

In [ ]:
# Генерация предсказаний
test_predictions = ranker.predict(test_pool)
test_df['predicted_score'] = test_predictions

print("✓ Предсказания готовы!")
print(f"\nСтатистика предсказанных scores:")
print(test_df['predicted_score'].describe())

## 13. Генерация топ-N рекомендаций для каждого query

In [ ]:
# Для каждого query берем топ-10 айтемов
def get_top_n_per_query(df, query_col, item_col, score_col, n=10):
    """
    Получить топ-N айтемов для каждого query
    """
    top_n = (
        df.sort_values([query_col, score_col], ascending=[True, False])
        .groupby(query_col)
        .head(n)
    )
    
    return top_n

top_recommendations = get_top_n_per_query(
    test_df, QUERY_COL, ITEM_COL, 'predicted_score', n=10
)

print(f"\nТоп-10 рекомендаций для {top_recommendations[QUERY_COL].nunique()} queries")
print(f"Всего рекомендаций: {len(top_recommendations)}")

# Пример для одного query
sample_query_id = test_df[QUERY_COL].iloc[0]
sample_recs = top_recommendations[top_recommendations[QUERY_COL] == sample_query_id]
print(f"\nПример рекомендаций для query {sample_query_id}:")
print(sample_recs[[ITEM_COL, 'predicted_score']].to_string(index=False))

## 14. Submission

In [ ]:
# Вариант 1: Все пары с predicted scores
submission_full = pd.DataFrame({
    'user_id': test_df[QUERY_COL],
    'item_id': test_df[ITEM_COL],
    'predicted_score': test_df['predicted_score']
})

submission_full.to_csv('catboost_ranker_full_submission.csv', index=False)
print("✓ Full submission сохранен!")

# Вариант 2: Только топ-10 для каждого query
submission_top10 = pd.DataFrame({
    'user_id': top_recommendations[QUERY_COL],
    'item_id': top_recommendations[ITEM_COL],
    'predicted_score': top_recommendations['predicted_score']
})

submission_top10.to_csv('catboost_ranker_top10_submission.csv', index=False)
print("✓ Top-10 submission сохранен!")

print("\nПример submission:")
print(submission_top10.head(10))

## 15. Сохранение модели

In [ ]:
# Сохранение модели
ranker.save_model('catboost_ranker_model.cbm')
print("✓ Модель сохранена!")

# Загрузка модели
# loaded_ranker = cb.CatBoost()
# loaded_ranker.load_model('catboost_ranker_model.cbm')